In [1]:
from xgboost import XGBRegressor
import pandas as pd
from sklearn.metrics import r2_score
import numpy as np

In [3]:
df=pd.read_csv('moscow_data.csv').set_index('TID').join(
    pd.read_csv('org_features.csv').set_index('TID')).fillna(0).join(
    pd.read_csv('target_nov_sept.csv').set_index('TID')[['target_sept']])
df=df[df.target_sept.notnull()]
x=df[df.columns[:-1]]
x=x.apply(lambda c: (c-c.mean())/c.std())
y=df.target_sept

In [5]:
mask=np.ones(len(x), dtype=bool)
cv_index=np.zeros(len(x), dtype=int)
num_col=len(x.columns)-1
cur_index=0
while np.sum(mask)>0 and num_col>len(x.columns)-1-35:
    new_mask=np.logical_and(mask, x[x.columns[num_col]]>0)
    new_elements=np.where(new_mask, new_mask.astype(int).cumsum()+cur_index, 0)
    mask=np.where(np.logical_not(new_mask), mask, False)
    cur_index+=len(new_mask)
    cv_index+=new_elements
    num_col-=1
cv_index+=np.where(mask, mask.astype(int).cumsum()+cur_index, 0)

In [7]:
y_actual=[]
y_pred=[]
for i in range(5):
    test=cv_index%5==i
    train=np.logical_not(test)
    y_actual.append(y[test])
    mod=XGBRegressor(verbosity=0, eta=0.015, max_depth=3)
    y_pred.append(mod.fit(x[train], y[train]).predict(x[test]))
r2_score(np.concatenate(y_actual), np.concatenate(y_pred))

-0.005389011221891948